In [7]:
# criar o navegador
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()

In [8]:
# abrir a página index (entrar no site da busca jurídica)
import os

file_path = os.getcwd()
file = file_path + r"\index.html"

In [9]:
# importar a base de dados
import pandas as pd

database = pd.read_excel('Processos.xlsx')
display(database)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,NaN
1,João,Lawyer Alon,EB3792,Rio de Janeiro,NaN
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,NaN
3,Carol,Amanda,PC5197,São Paulo,NaN


In [10]:
from selenium.webdriver import ActionChains
import time

for row in database.index:
  
    # para cada processo (linha da tabela)
    driver.get(file)


    # abrir a lista de cidades
    button = driver.find_element(By.CLASS_NAME, 'dropdown-menu')
    ActionChains(driver).move_to_element(button).perform()

    city = database.loc[row, 'Cidade']

    # selecionando o cidade
    driver.find_element(By.PARTIAL_LINK_TEXT, city).click()
    # mudar para a nova aba
    original_tab = driver.window_handles[0]
    index = 1 + row
    new_tab = driver.window_handles[index]

    driver.switch_to.window(new_tab)

    # preencher o formulário com os dados de busca
    driver.find_element(By.ID, 'nome').send_keys(database.loc[row, 'Nome'])
    driver.find_element(By.ID, 'advogado').send_keys(database.loc[row, 'Advogado'])
    driver.find_element(By.ID, 'numero').send_keys(database.loc[row, 'Processo'])

    # clicar em pesquisar
    driver.find_element(By.CLASS_NAME, 'registerbtn').click()

    # confirmar a pesquisa
    alert = driver.switch_to.alert
    alert.accept()
    # esperar o resultado da pesquisa e agir de acordo com o resultado
    while True:
        try:
            alert = driver.switch_to.alert
            break
        except:
            time.sleep(1)
    text_alert = alert.text
    if 'Processo encontrado com sucesso' in text_alert:
        alert.accept()
        database.loc[row, 'Status'] = 'Encontrado'
    else:
        alert.accept()
        database.loc[row, 'Status'] = 'Não encontrado'

C:\Users\Dell\AppData\Local\Temp\ipykernel_1084\3193900267.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Encontrado' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  database.loc[row, 'Status'] = 'Encontrado'


In [11]:
driver.quit()
display(database)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,Encontrado
1,João,Lawyer Alon,EB3792,Rio de Janeiro,Não encontrado
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,Não encontrado
3,Carol,Amanda,PC5197,São Paulo,Encontrado
